# Lab 11: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [6]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from localization_extras import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-04-24 14:37:43,211 | INFO     |: Logger demo_notebook.log already initialized.
2024-04-24 14:37:43,212 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

2024-04-24 14:37:43,218 | ERROR    |: Simulator is already running
2024-04-24 14:37:43,219 | ERROR    |: Plotter is already running


In [7]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2024-04-24 14:37:49,261 | INFO     |:  | Number of observations per grid cell: 18
2024-04-24 14:37:49,262 | INFO     |:  | Precaching Views...
2024-04-24 14:37:53,057 | INFO     |:  | Precaching Time: 3.795 secs
2024-04-24 14:37:53,058 | INFO     |: Initializing beliefs with a Uniform Distribution
2024-04-24 14:37:53,059 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run the Bayes Filter
The cells below utilizes the member functions of class **Localization** (defined in [localization_extras.py](../localization_extras.py)) in each iteration of the Bayes filter algorithm to localize the robot in the grid map. <br>

In [8]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2024-04-24 14:37:58,607 | INFO     |: Initializing beliefs with a Uniform Distribution
2024-04-24 14:37:58,608 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2024-04-24 14:38:01,565 | INFO     |: Update Step
2024-04-24 14:38:01,569 | INFO     |:      | Update Time: 0.003 secs
2024-04-24 14:38:01,571 | INFO     |: ---------- UPDATE STATS -----------
2024-04-24 14:38:01,574 | INFO     |: GT index      : (6, 4, 9)
2024-04-24 14:38:01,576 | INFO     |: Bel index     : (5, 4, 9) with prob = 0.9999999
2024-04-24 14:38:01,576 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2024-04-24 14:38:01,578 | INFO     |: GT            : (0.000, 0.000, 360.000)
2024-04-24 14:38:01,580 | INFO     |: Belief        : (0.000, 0.000, 10.000)
2024-04-24 14:38:01,581 | INFO     |: POS ERROR     : (-0.000, -0.000, 350.000)
2024-04-24 14:38:01,584 | INFO     |: ---------- UPDATE STATS -----------


In [9]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------
2024-04-24 14:38:06,798 | INFO     |: Prediction Step
2024-04-24 14:38:06,849 | INFO     |:  | Prediction Time: 0.050 secs
2024-04-24 14:38:06,850 | INFO     |: ---------- PREDICTION STATS -----------
2024-04-24 14:38:06,864 | INFO     |: GT index         : (6, 3, 7)
2024-04-24 14:38:06,865 | INFO     |: Prior Bel index  : (7, 4, 8) with prob = 0.0771558
2024-04-24 14:38:06,866 | INFO     |: POS ERROR        : (-0.328, -0.087, -29.725)
2024-04-24 14:38:06,867 | INFO     |: ---------- PREDICTION STATS -----------
2024-04-24 14:38:09,831 | INFO     |: Update Step
2024-04-24 14:38:09,836 | INFO     |:      | Update Time: 0.004 secs
2024-04-24 14:38:09,837 | INFO     |: ---------- UPDATE STATS -----------
2024-04-24 14:38:09,855 | INFO     |: GT index      : (6, 3, 7)
2024-04-24 14:38:09,856 | INFO     |: Bel index     : (6, 4, 6) with prob = 1.0
2024-04-24 14:38:09,857 | INFO     |: Bel_bar prob at index = 0.0003764664743832955
2024-04-24 14:38:09,8